In [1]:
from langchain.llms import GooglePalm 
from langchain.document_loaders import UnstructuredURLLoader 
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS 
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.chains import RetrievalQA
from keys import api_keys   


In [3]:
llm = GooglePalm(google_api_key=api_keys,temperature=0.6)

In [4]:
# first load the content from the website 
urls = [
    "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)",
    "https://cleartax.in/s/world-gdp-ranking-list", 
    "https://www.focus-economics.com/countries/bangladesh/"]
loader=UnstructuredURLLoader(urls=urls)
data=loader.load()
data

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'}, page_content='Toggle the table of contents\n\nList of countries by GDP (nominal)\n\n76 languages\n\nAfrikaans\n\nالعربية\n\nAsturianu\n\nAzərbaycanca\n\nتۆرکجه\n\nবাংলা\n\n閩南語 / Bân-lâm-gú\n\nБашҡортса\n\nБеларуская\n\nभोजपुरी\n\nБългарски\n\nCatalà\n\nČeština\n\nCymraeg\n\nDansk\n\nDeutsch\n\nΕλληνικά\n\nEspañol\n\nEsperanto\n\nفارسی\n\nFrançais\n\nGaeilge\n\nGalego\n\n한국어\n\nՀայերեն\n\nहिन्दी\n\nIdo\n\nIlokano\n\nBahasa Indonesia\n\nÍslenska\n\nItaliano\n\nქართული\n\nҚазақша\n\nKurdî\n\nLatviešu\n\nLietuvių\n\nMagyar\n\nМакедонски\n\nമലയാളം\n\nBahasa Melayu\n\nМонгол\n\nNederlands\n\n日本語\n\nNorsk bokmål\n\nNorsk nynorsk\n\nOʻzbekcha / ўзбекча\n\nپنجابی\n\nپښتو\n\nPlattdüütsch\n\nPolski\n\nPortuguês\n\nRomână\n\nРусский\n\nShqip\n\nසිංහල\n\nSimple English\n\nSlovenčina\n\nSlovenščina\n\nکوردی\n\nСрпски / srpski\n\nSrpskohrvatski / српскохрватски\n\nSunda\n\nSuomi\n\nSvenska\n\nTagalog\n\

In [5]:
#initiate the splitter 
splitter=RecursiveCharacterTextSplitter(
    separators=["\n\n",".","\n"],
    chunk_size=1000,
    chunk_overlap=200
)
splitted_docs = splitter.split_documents(data)

In [6]:
#initiate the embedding 
google_embedding = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001", google_api_key=api_keys)
#initate the vector database 
vectdb = FAISS.from_documents(splitted_docs, google_embedding)


In [7]:

chain = RetrievalQA.from_llm(
    llm=llm, 
    retriever=vectdb.as_retriever(),    
    input_key='query',
    return_source_documents=True)


In [55]:
quest = "Give a overal summary of the condition of bangladesh economy"
result = chain({'query': quest})

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Give a overal summary of the condition of bangladesh economy"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Give a overal summary of the condition of bangladesh economy",
  "context": "Context:\n.4 5.7 5.3 6.1 8.7 Inflation (CPI, ann. var. %, aop) 5.5 5.6 5.7 5.5 7.7 Exchange Rate (BDT per USD, eop) 83.25 84.95 84.72 84.86 103.02 Exchange Rate (BDT per USD, aop) 83.73 84.42 84.88 84.90 92.65 Current Account Balance (USD bn) -9.6 -4.5 -5.4 -4.6 -18.6 Current Account Balance (% of GDP) -3.0 -1.3 -1.5 -1.1 -4.1 Merchandise Trade Balance (USD bn) -16.0 -15.6 -16.4 -32.5 -28.9 Merchandise Exports (USD bn) 38.6 39.0 32.4 42.0 51.0 Merchandise Imports (USD bn) 54.6 54.6 48.8 74.5 79.9 Merchandise Exports (ann. var. %) 11.0 1.1 -17.0 

In [60]:
result['source_documents'][0].metadata['source']

'https://www.focus-economics.com/countries/bangladesh/'